In [1]:
from datasets import load_dataset
 
ds = load_dataset("Hemg/Deepfake-Audio-Dataset")

c:\Users\ihadi\Desktop\Semestre 9\Explainaibility_ai\Projet\Xai-unified-interface\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import tensorflow as tf
import tf_keras as keras

def load_unified_model(model_name):
    # Les architectures pour Repo 1 (Audio) et Repo 2 (Image)
    if "VGG16" in model_name:
        return keras.applications.vgg16.VGG16(weights='imagenet')
    elif "ResNet" in model_name or "AlexNet" in model_name:
        return keras.applications.resnet50.ResNet50(weights='imagenet')
    elif "MobileNet" in model_name or "DenseNet" in model_name:
        return keras.applications.densenet.DenseNet121(weights='imagenet')
    return keras.applications.vgg16.VGG16(weights='imagenet')

c:\Users\ihadi\Desktop\Semestre 9\Explainaibility_ai\Projet\Xai-unified-interface\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


# Training of AlexNet

In [3]:
alex_net = load_unified_model("AlexNet")

In [4]:
from tf_keras import layers, models

# alex_net est déjà chargé via load_unified_model("AlexNet")
# Rappel : C'est techniquement un ResNet50 à cause de ta fonction.

# 1. On récupère la sortie de l'avant-dernière couche (juste avant les 1000 classes)
# Pour ResNet50 de Keras, la dernière couche est souvent 'predictions'
# On remonte donc d'un cran.
x_base = alex_net.layers[-2].output 

# 2. On ajoute tes couches personnalisées
x = layers.Flatten()(x_base)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dropout(0.5)(x)

# 3. Sortie binaire (Tumeur ou pas Tumeur)
outputs = layers.Dense(1, activation='sigmoid')(x)

# 4. On crée le nouveau modèle en utilisant alex_net.input
# On utilise keras.models (venant de tf_keras) pour rester cohérent
new_alex_net = models.Model(inputs=alex_net.input, outputs=outputs)

# 5. Compilation
new_alex_net.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [5]:
dense_net = load_unified_model("DenseNet")

In [6]:
from datasets import load_dataset

ds = load_dataset("gymprathap/Brain-Tumor-Segmentation")

In [7]:
ds

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 7858
    })
})

In [8]:
import tensorflow as tf
from PIL import Image
import numpy as np

IMG_SIZE = 224 # Taille cible pour l'entraînement

def preprocess_images(examples):
    images = []
    masks = []
    
    for img, mask in zip(examples['image'], examples['label']):
        # Redimensionner et convertir l'image en RGB ou Niveaux de gris
        img = img.resize((IMG_SIZE, IMG_SIZE)).convert("RGB")
        # Redimensionner le masque (interpolation 'NEAREST' pour garder les labels intacts)
        mask = mask.resize((IMG_SIZE, IMG_SIZE), resample=Image.NEAREST)
        
        images.append(np.array(img) / 255.0) # Normalisation [0, 1]
        masks.append(np.array(mask))
        
    return {"pixel_values": images, "labels": masks}

# Appliquer la transformation
processed_ds = ds.with_transform(preprocess_images)

In [9]:
processed_ds

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 7858
    })
})

In [10]:
import numpy as np
from PIL import Image

IMG_SIZE = 224
x_list = []
y_list = []

print("Extraction et redimensionnement en cours...")

# On itère sur le dataset original 'ds' pour éviter les transformations entremêlées
for i in range(len(ds['train'])):
    item = ds['train'][i]
    
    # Traitement Image (X)
    img = item['image'].resize((IMG_SIZE, IMG_SIZE)).convert("RGB")
    x_list.append(np.array(img) / 255.0)
    
    # Traitement Label (Y) : On transforme le masque en 0 ou 1
    mask = np.array(item['label'])
    has_tumor = 1 if np.max(mask) > 0 else 0
    y_list.append(has_tumor)

# Conversion en tableaux NumPy
x_train = np.array(x_list, dtype=np.float32)
y_train = np.array(y_list, dtype=np.int32)

print(f"Extraction terminée : {len(x_train)} images prêtes.")

Extraction et redimensionnement en cours...
Extraction terminée : 7858 images prêtes.


In [11]:
import tensorflow as tf

# Création du dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

# Configuration : Shuffle + Batch
# Un buffer de 1000 est suffisant pour bien mélanger
train_dataset = train_dataset.shuffle(1000).batch(16).prefetch(tf.data.AUTOTUNE)

print("Dataset TensorFlow prêt !")

Dataset TensorFlow prêt !


In [17]:
# Calcul du nombre de batches
import math
total_images = 7858
batch_size = 16
steps_per_epoch = math.ceil(total_images / batch_size)

# Lancement de l'entraînement avec le nombre de pas défini
history = new_alex_net.fit(
    train_dataset,
    epochs=3,
    steps_per_epoch=steps_per_epoch
)

Epoch 1/3
492/492 [==============================] - 2337s 5s/step - loss: 0.0222 - accuracy: 0.9964
Epoch 2/3
492/492 [==============================] - 1805s 4s/step - loss: 0.0135 - accuracy: 0.9975
Epoch 3/3
492/492 [==============================] - 1159s 2s/step - loss: 0.0219 - accuracy: 0.9971


In [18]:
new_alex_net.save("alexnet_brain_tumor_model.h5")

# Training of dense net

In [14]:
from tf_keras import layers, models

# alex_net est déjà chargé via load_unified_model("AlexNet")
# Rappel : C'est techniquement un ResNet50 à cause de ta fonction.

# 1. On récupère la sortie de l'avant-dernière couche (juste avant les 1000 classes)
# Pour ResNet50 de Keras, la dernière couche est souvent 'predictions'
# On remonte donc d'un cran.
x_base = dense_net.layers[-2].output 

# 2. On ajoute tes couches personnalisées
x = layers.Flatten()(x_base)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dropout(0.5)(x)

# 3. Sortie binaire (Tumeur ou pas Tumeur)
outputs = layers.Dense(1, activation='sigmoid')(x)

# 4. On crée le nouveau modèle en utilisant dense_net.input
# On utilise keras.models (venant de tf_keras) pour rester cohérent
new_dense_net = models.Model(inputs=dense_net.input, outputs=outputs)

# 5. Compilation
new_dense_net.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [15]:
# Lancer l'entraînement
history = new_dense_net.fit(
    train_dataset,
    epochs=7
)

Epoch 1/7
492/492 [==============================] - 1040s 2s/step - loss: 0.0374 - accuracy: 0.9941
Epoch 2/7
492/492 [==============================] - 988s 2s/step - loss: 0.0305 - accuracy: 0.9945
Epoch 3/7
492/492 [==============================] - 985s 2s/step - loss: 0.0290 - accuracy: 0.9948
Epoch 4/7
492/492 [==============================] - 989s 2s/step - loss: 0.0270 - accuracy: 0.9957
Epoch 5/7
492/492 [==============================] - 990s 2s/step - loss: 0.0231 - accuracy: 0.9953
Epoch 6/7
492/492 [==============================] - 989s 2s/step - loss: 0.0192 - accuracy: 0.9966
Epoch 7/7
492/492 [==============================] - 987s 2s/step - loss: 0.0214 - accuracy: 0.9961


In [ ]:
new_dense_net.save("densenet_brain_tumor_model.h5")